In [1]:
%load_ext autoreload
%autoreload 2

# Necessary imports
import sys

import torch
import pandas as pd
pd.set_option('display.max_rows', 200)

sys.path.insert(0, "../../..")
from batchflow.models.torch import *

# Setup

In [2]:
DEVICE = torch.device('cuda:0')

BASE_SHAPE = (16, 32, 256, 256)
SHAPES = [(BASE_SHAPE[0], BASE_SHAPE[1] * (2 ** i),
           BASE_SHAPE[2] // (2 ** i), BASE_SHAPE[3] // (2 ** i))
          for i in range(5)]
SHAPES

[(16, 32, 256, 256),
 (16, 64, 128, 128),
 (16, 128, 64, 64),
 (16, 256, 32, 32),
 (16, 512, 16, 16)]

# Conv

In [3]:
%%time
stats_list = []
for shape in SHAPES:
    inputs = torch.rand(*shape, device=DEVICE)

    modules = {
        'conv_1x1': Block(inputs=inputs, layout='c', channels='same', kernel_size=1),
        'conv_1x1_bias': Block(inputs=inputs, layout='c', channels='same', kernel_size=1, bias=True),

        'conv_3x3': Block(inputs=inputs, layout='c', channels='same', kernel_size=3),
        'conv_3x3_bias': Block(inputs=inputs, layout='c', channels='same', kernel_size=3, bias=True),
        'conv_3x3_depthwise': Block(inputs=inputs, layout='w', channels='same', kernel_size=3),
        'conv_3x3_depthwise_bias': Block(inputs=inputs, layout='w', channels='same', kernel_size=3, bias=True),
        'conv_3x3_dilation3': Block(inputs=inputs, layout='c', channels='same', kernel_size=3, dilation=3),

        'K_1357': Block(inputs=inputs, layout='K', channels='same', kernel_size=(1, 3, 5, 7), dilation=1),
        'K_3x3_dilation1357': Block(inputs=inputs, layout='K', channels='same', kernel_size=(3, 3, 3, 3),
                                    dilation=(1, 3, 5, 7)),
    }

    for name, module in modules.items():
        for amp in [False, True]:
            stats = get_module_performance(module, inputs=inputs, n_repeats=100, warmup=10, amp=amp)
            stats_list.append({'module_name': name, 'amp': amp, 'shape': shape, **stats})


dataframe = pd.DataFrame(stats_list).set_index(['shape', 'module_name', 'amp'])
dataframe

CPU times: user 25.9 s, sys: 16.4 s, total: 42.2 s
Wall time: 43 s


parameters  \
shape              module_name             amp                 
(16, 32, 256, 256) conv_1x1                False        1024   
                                           True         1024   
                   conv_1x1_bias           False        1056   
                                           True         1056   
                   conv_3x3                False        9216   
                                           True         9216   
                   conv_3x3_bias           False        9248   
                                           True         9248   
                   conv_3x3_depthwise      False         288   
                                           True          288   
                   conv_3x3_depthwise_bias False         320   
                                           True          320   
                   conv_3x3_dilation3      False        9216   
                                           True         9216   
                   K_1357                  False       21504   
                                           True        21504   
                   K_3x3_dilation1357      False        9216   
                                           True         9216   
(16, 64, 128, 128) conv_1x1                False        4096   
                                           True         4096   
                   conv_1x1_bias           False        4160   
                                           True         4160   
                   conv_3x3                False       36864   
                                           True        36864   
                   conv_3x3_bias           False       36928   
                                           True        36928   
                   conv_3x3_depthwise      False         576   
                                           True          576   
                   conv_3x3_depthwise_bias False         640   
                                           True          640   
                   conv_3x3_dilation3      False       36864   
                                           True        36864   
                   K_1357                  False       86016   
                                           True        86016   
                   K_3x3_dilation1357      False       36864   
                                           True        36864   
(16, 128, 64, 64)  conv_1x1                False       16384   
                                           True        16384   
                   conv_1x1_bias           False       16512   
                                           True        16512   
                   conv_3x3                False      147456   
                                           True       147456   
                   conv_3x3_bias           False      147584   
                                           True       147584   
                   conv_3x3_depthwise      False        1152   
                                           True         1152   
                   conv_3x3_depthwise_bias False        1280   
                                           True         1280   
                   conv_3x3_dilation3      False      147456   
                                           True       147456   
                   K_1357                  False      344064   
                                           True       344064   
                   K_3x3_dilation1357      False      147456   
                                           True       147456   
(16, 256, 32, 32)  conv_1x1                False       65536   
                                           True        65536   
                   conv_1x1_bias           False       65792   
                                           True        65792   
                   conv_3x3                False      589824   
                                           True       589824   
                   conv_3x3_bias           False      590080   
                  

In [4]:
# Amp speeds up things, but does not change memory required!
dataframe.groupby('amp').mean()

,parameters,"forward time mean, ms","forward time std, ms","forward memory, MB","backward time mean, ms","backward time std, ms","backward memory, MB","time total, ms"
amp,,,,,,,,
False,458281.955556,1.764360,0.012868,126.101997,3.525029,0.052498,109.933887,598.363288
True,458281.955556,1.158195,0.009440,126.092274,1.305790,0.047201,109.944998,296.948743


# Normalization

In [5]:
%%time
stats_list = []
for shape in SHAPES:
    inputs = torch.rand(*shape, device=DEVICE)

    modules = {
        'batchnorm': Block(inputs=inputs, layout='n', normalization_type='batch'),
        'instancenorm': Block(inputs=inputs, layout='n', normalization_type='instance'),
        'instancenorm_affine': Block(inputs=inputs, layout='n', normalization_type='instance',
                                     normalization={'affine': True}),
        'layernorm': Block(inputs=inputs, layout='n', normalization_type='layer'),
    }

    for name, module in modules.items():
        module = module.to(inputs.device)
        for amp in [False, True]:
            stats = get_module_performance(module, inputs=inputs, n_repeats=100, warmup=10, amp=amp)
            stats_list.append({'module_name': name, 'amp': amp, 'shape': shape, **stats})


dataframe = pd.DataFrame(stats_list).set_index(['shape', 'module_name', 'amp'])
dataframe

CPU times: user 4.02 s, sys: 1.7 s, total: 5.72 s
Wall time: 5.86 s


parameters  \
shape              module_name         amp                 
(16, 32, 256, 256) batchnorm           False          64   
                                       True           64   
                   instancenorm        False           0   
                                       True            0   
                   instancenorm_affine False          64   
                                       True           64   
                   layernorm           False          64   
                                       True           64   
(16, 64, 128, 128) batchnorm           False         128   
                                       True          128   
                   instancenorm        False           0   
                                       True            0   
                   instancenorm_affine False         128   
                                       True          128   
                   layernorm           False         128   
                                       True          128   
(16, 128, 64, 64)  batchnorm           False         256   
                                       True          256   
                   instancenorm        False           0   
                                       True            0   
                   instancenorm_affine False         256   
                                       True          256   
                   layernorm           False         256   
                                       True          256   
(16, 256, 32, 32)  batchnorm           False         512   
                                       True          512   
                   instancenorm        False           0   
                                       True            0   
                   instancenorm_affine False         512   
                                       True          512   
                   layernorm           False         512   
                                       True          512   
(16, 512, 16, 16)  batchnorm           False        1024   
                                       True         1024   
                   instancenorm        False           0   
                                       True            0   
                   instancenorm_affine False        1024   
                                       True         1024   
                   layernorm           False        1024   
                                       True         1024   

                                              forward time mean, ms  \
shape              module_name         amp                            
(16, 32, 256, 256) batchnorm           False               1.032504   
                                       True                1.007319   
                   instancenorm        False               0.893964   
                                       True                0.891875   
                   instancenorm_affine False               0.915812   
                                       True                0.916036   
                   layernorm           False               3.103204   
                                       True                3.100700   
(16, 64, 128, 128) batchnorm           False               0.542812   
                                       True                0.540575   
                   instancenorm        False               0.442079   
                                       True                0.441640   
                   instancenorm_affine False               0.507103   
                                       True                0.566130   
                   layernorm           False               1.575159   
                                       True                1.574473   
(16, 128, 64, 64)  batchnorm           False               0.312148   
                                       True                0.346022   
                   instancenorm        False               0.249267   
                                  

# Named blocks

In [6]:
def simplegate(x):
    return x[:, 0::2] * x[:, 1::2]

In [7]:
%%time
stats_list = []
for shape in SHAPES:
    inputs = torch.rand(*shape, device=DEVICE)

    modules = {
        'resblock': ResBlock(inputs=inputs, attention='se'),
        'resblock_nrep2': ResBlock(inputs=inputs, n_reps=2, attention='se'),
        'resblock_bottleneck4': BottleneckBlock(inputs=inputs, bottleneck=4, attention='se'),
        'resblock_groups8': ResBlock(inputs=inputs, groups=8, attention='se'),
        'resblock_bottleneck4groups8': BottleneckBlock(inputs=inputs, bottleneck=4, groups=8, attention='se'),

        'convnext_layernorm': ConvNeXtBlock(inputs=inputs, normalization_type='layer'),
        'convnext_batchnorm': ConvNeXtBlock(inputs=inputs, normalization_type='batch'),

        'afblock': Block(inputs=inputs, layout='RncwaSc! Rncac!',
                         channels=['2 * same', 'same', 'same', '2 * same', 'same'],
                         kernel_size=[1, 3, 1, 1, 1], bias=True,
                         activation=simplegate, attention='se', self_attention={'bias': True},
                         branch_end={'drop_path': 0.0, 'layer_scale': 0.001})
    }

    for name, module in modules.items():
        module = module.to(inputs.device)
        for amp in [False, True]:
            stats = get_module_performance(module, inputs=inputs, n_repeats=100, warmup=10, amp=amp)
            stats_list.append({'module_name': name, 'amp': amp, 'shape': shape, **stats})


dataframe = pd.DataFrame(stats_list).set_index(['shape', 'module_name', 'amp'])
dataframe

CPU times: user 1min 44s, sys: 57.3 s, total: 2min 41s
Wall time: 2min 43s


parameters  \
shape              module_name                 amp                 
(16, 32, 256, 256) resblock                    False       19072   
                                               True        19072   
                   resblock_nrep2              False       38144   
                                               True        38144   
                   resblock_bottleneck4        False       42880   
                                               True        42880   
                   resblock_groups8            False        2944   
                                               True         2944   
                   resblock_bottleneck4groups8 False       10624   
                                               True        10624   
                   convnext_layernorm          False       10048   
                                               True        10048   
                   convnext_batchnorm          False       10048   
                                               True        10048   
                   afblock                     False        7720   
                                               True         7720   
(16, 64, 128, 128) resblock                    False       76032   
                                               True        76032   
                   resblock_nrep2              False      152064   
                                               True       152064   
                   resblock_bottleneck4        False      170752   
                                               True       170752   
                   resblock_groups8            False       11520   
                                               True        11520   
                   resblock_bottleneck4groups8 False       41728   
                                               True        41728   
                   convnext_layernorm          False       36480   
                                               True        36480   
                   convnext_batchnorm          False       36480   
                                               True        36480   
                   afblock                     False       28752   
                                               True        28752   
(16, 128, 64, 64)  resblock                    False      303616   
                                               True       303616   
                   resblock_nrep2              False      607232   
                                               True       607232   
                   resblock_bottleneck4        False      681472   
                                               True       681472   
                   resblock_groups8            False       45568   
                                               True        45568   
                   resblock_bottleneck4groups8 False      165376   
                                               True       165376   
                   convnext_layernorm          False      138496   
                                               True       138496   
                   convnext_batchnorm          False      138496   
                                               True       138496   
                   afblock                     False      110752   
                                               True       110752   
(16, 256, 32, 32)  resblock                    False     1213440   
                                               True      1213440   
                   resblock_nrep2              False     2426880   
                                               True      2426880   
                   resblock_bottleneck4        False     2722816   
                                               True      2722816   
                   resblock_groups8            False      181248   
                                               True       181248   
                   resblock_bottleneck4groups8 False      658432   
                                          